In [2]:
import pandas as pd;
import numpy  as np;
import scipy.integrate;
import matplotlib.pyplot as plt;
import copy;
from   sklearn.metrics       import mean_squared_error, mean_absolute_error;
from   sklearn.linear_model  import LinearRegression, BayesianRidge;
from   sklearn.tree          import DecisionTreeRegressor;
from   sklearn.preprocessing import PolynomialFeatures

In [29]:
# ** IMPORTING VALIDATION LIBRARY **
from validation.unit import data_assumptions, model_metrics

from validation.integration import (
   find_outliers, find_duplicates, find_balance, find_correlation
   ,validate_dataset, test_representative 
)
# **********************************

In [4]:
dir_str = "test_repos/Regression-Analysis-of-COVID-19/covid_19_data.csv";
df_orig = pd.read_csv(dir_str, error_bad_lines = False);

C:\Users\Caleb\AppData\Local\Temp\ipykernel_44712\3728841523.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_orig = pd.read_csv(dir_str, error_bad_lines = False);


    Using the validation functions to test the current state of the data

In [34]:
data_assumptions(df_orig)

,index,null_count,Data_Types
0,SNo,0.0,0
1,ObservationDate,0.0,0
2,Province/State,4956.0,"[<class 'str'>, <class 'float'>]"
3,Country/Region,0.0,0
4,Last Update,0.0,0
5,Confirmed,0.0,0
6,Deaths,0.0,0
7,Recovered,0.0,0


In [44]:
validate_dataset(df_orig, categorical_threshold=0.15, corr_method="pearson"
                 , threshold = 0.75, data_type="df")

D:\PycharmProjects\ValidationProject\validation\integration.py:100: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = data_df.corr(method=corr_method)


{'outliers': Empty DataFrame
 Columns: [Column, Value, Index, Issue]
 Index: [],
 'duplicates': Empty DataFrame
 Columns: [Column, Value, Index, Issue]
 Index: [],
 'correlation':    Attribute_1 Attribute_2  Absolute_Correlation
 6    Confirmed      Deaths              0.873705
 7    Confirmed   Recovered              0.753910
 9       Deaths   Confirmed              0.873705
 13   Recovered   Confirmed              0.753910}

In [45]:
x = validate_dataset(df_orig, categorical_threshold=0.15, corr_method="pearson"
                 , threshold = 0.75, data_type="df")

D:\PycharmProjects\ValidationProject\validation\integration.py:100: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = data_df.corr(method=corr_method)


In [46]:
x["correlation"]

,Attribute_1,Attribute_2,Absolute_Correlation
6,Confirmed,Deaths,0.873705
7,Confirmed,Recovered,0.753910
9,Deaths,Confirmed,0.873705
13,Recovered,Confirmed,0.753910


    ***** 

In [5]:
df = copy.deepcopy(df_orig);
del df['Last Update'];
date_lst  = df.ObservationDate.unique();
date_dict = {}
for i in range(len(date_lst)):
    df_temp = df.loc[df['ObservationDate'] == date_lst[i]];
    date_dict[date_lst[i]] = df_temp;    

In [6]:
date_tot_tup_dict = {};
for date, df in date_dict.items():
    tup_temp = (df['Confirmed'].sum(), df['Deaths'].sum(), df['Recovered'].sum())
    date_tot_tup_dict[date] = tup_temp;

In [7]:
df_date_tots = pd.DataFrame(date_tot_tup_dict);
df_date_tots = df_date_tots.transpose();
df_date_tots.columns = ['Confirmed', 'Deaths', 'Recovered'];

In [8]:
df_date_tots['Closed Cases'] = df_date_tots['Deaths'] + df_date_tots['Recovered'];

In [9]:
df_date_tots['Active Cases'] = df_date_tots['Confirmed'] - df_date_tots['Closed Cases']

In [10]:
growth_len_lst = len(date_lst) - 1; # 69
confirmed_lst = df_date_tots["Confirmed"];
death_lst = df_date_tots["Deaths"];
recovered_lst = df_date_tots["Recovered"];
confGF_lst = []
deathsGF_lst = []
recovGF_lst = []
for i in range(growth_len_lst):
    confirmedGF = confirmed_lst[i+1] / confirmed_lst[i] ;
    confGF_lst.append(confirmedGF)
    deathGF = death_lst[i+1] / death_lst[i] ;
    deathsGF_lst.append(deathGF)
    recoveredGF = recovered_lst[i+1] / recovered_lst[i] ;
    recovGF_lst.append(recoveredGF);

In [11]:
days_since_lst = [];
for i in range(len(date_lst)):
    days_since_lst.append(i);
df_date_tots["Days Since:"] = days_since_lst;
df_date_tots = df_date_tots[["Days Since:", "Confirmed", "Deaths", "Recovered", "Active Cases", "Closed Cases"]];

In [12]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV;
from sklearn.preprocessing import StandardScaler;
X = np.array(df_date_tots["Confirmed"])
y = np.array(df_date_tots["Days Since:"]).reshape(-1,1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.15,
                                                    shuffle = False)

In [15]:
lin_model = LinearRegression(fit_intercept = False);
lin_model.fit(y_train, X_train);
test_lin_pred = lin_model.predict(y_test)
lin_pred = lin_model.predict(y)

In [16]:
poly = PolynomialFeatures(degree=5)
poly_y_train = poly.fit_transform(y_train)
poly_y_test  = poly.fit_transform(y_test)
poly_y = poly.fit_transform(y)

In [18]:
linear_model = LinearRegression(fit_intercept=False)
linear_model.fit(poly_y_train, X_train)
test_linear_pred = linear_model.predict(poly_y_test)
linear_pred = linear_model.predict(poly_y)


In [20]:
tol      = [1e-4, 1e-3, 1e-2]
alpha_1  = [1e-7, 1e-6, 1e-5, 1e-4]
alpha_2  = [1e-7, 1e-6, 1e-5, 1e-4]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4]

bayesian_grid = {'tol': tol,
                 'alpha_1': alpha_1,
                 'alpha_2' : alpha_2,
                 'lambda_1': lambda_1,
                 'lambda_2' : lambda_2 }

bayesian = BayesianRidge(fit_intercept=False)
bayesian_search = RandomizedSearchCV(bayesian,
                                     bayesian_grid,
                                     scoring='neg_mean_squared_error',
                                     cv=3,
                                     return_train_score=True,
                                     n_jobs=-1,
                                     n_iter=40,
                                     verbose=1)

bayesian_search.fit(y_train, X_train);

Fitting 3 folds for each of 40 candidates, totalling 120 fits


In [21]:
y_test_ridge_pred = bayesian_search.predict(y_test);
y_ridge_pred = bayesian_search.predict(y);

In [22]:
bayesian_search_poly = RandomizedSearchCV(bayesian,
                                     bayesian_grid,
                                     scoring='neg_mean_squared_error',
                                     cv=3,
                                     return_train_score=True,
                                     n_jobs=-1,
                                     n_iter=40,
                                     verbose=1)


In [23]:
bayesian_search_poly.fit(poly_y_train, X_train);

Fitting 3 folds for each of 40 candidates, totalling 120 fits


In [24]:
bayesian_poly_confirmed = bayesian_search_poly.best_estimator_
test_poly_bayesian_pred = bayesian_poly_confirmed.predict(poly_y_test)
bayesian_poly_pred = bayesian_poly_confirmed.predict(poly_y)

In [25]:
from sklearn.svm import SVR
kernel = ['poly', 'sigmoid', 'rbf']
c = [0.01, 0.1, 1, 10, 100];
gamma = [0.01, 0.1, 1];
epsilon = [0.01, 0.1, 1];
shrinking = [True, False];
svm_grid = {'kernel' :kernel,
           'C' : c,
           'gamma':gamma,
           'epsilon': epsilon,
           'shrinking': shrinking}

svm = SVR();
svm_search = RandomizedSearchCV(svm, svm_grid, scoring = 'neg_mean_squared_error', cv=3,
                               return_train_score = True)


svm_search.fit(y_train, X_train) # note how y_train had to be put first

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': [0.01, 0.1, 1, 10, 100],
                                        'epsilon': [0.01, 0.1, 1],
                                        'gamma': [0.01, 0.1, 1],
                                        'kernel': ['poly', 'sigmoid', 'rbf'],
                                        'shrinking': [True, False]},
                   return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
y_pred_test = svm_search.predict(y_test)

In [27]:
y_pred = svm_search.predict(y)

In [28]:
DT_reg = DecisionTreeRegressor(random_state=0);
DT_reg.fit(y_train, X_train);
y_test_DT_pred = DT_reg.predict(y_test);
y_DT_pred = DT_reg.predict(y);